In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [2]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [3]:
print(train_df.shape)
print(test_df.shape)

(15035, 21)
(6468, 20)


## 변수 생성

In [4]:
#create column for new variable (one is enough because it's a binary categorical feature)
#if area>0 it gets 1, for area==0 it gets 0 
train_df['HasBsmt'] = pd.Series(len(train_df['sqft_basement']), index=train_df.index)
train_df['HasBsmt'] = 0
train_df.loc[train_df['sqft_basement']>0,'HasBsmt'] = 1

In [5]:
#create column for new variable (one is enough because it's a binary categorical feature)
#if area>0 it gets 1, for area==0 it gets 0 
test_df['HasBsmt'] = pd.Series(len(test_df['sqft_basement']), index=test_df.index)
test_df['HasBsmt'] = 0 
test_df.loc[test_df['sqft_basement']>0,'HasBsmt'] = 1

In [6]:
#transform data
train_df.loc[train_df['HasBsmt']==1,'sqft_basement'] = np.log(train_df['sqft_basement'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [7]:
#transform data
test_df.loc[test_df['HasBsmt']==1,'sqft_basement'] = np.log(test_df['sqft_basement'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [8]:
train_df = pd.get_dummies(train_df.loc[:,train_df.columns != 'date'])
test_df = pd.get_dummies(test_df.loc[:,test_df.columns != 'date'])

In [9]:
train_df['price'] = np.log(train_df['price'])

In [10]:
train_y = train_df['price']
train_X = train_df.loc[:, train_df.columns != 'price']
train_X = train_X.loc[:, train_X.columns != 'id']

In [11]:
print(train_X.columns)
print(len(train_X.columns))
print(test_df.columns)
print(len(test_df.columns))

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'HasBsmt'],
      dtype='object')
19
Index(['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'HasBsmt'],
      dtype='object')
20


In [12]:
print('train_X shape : ', train_X.shape)
print('test_df shape : ', test_df.shape)

train_X shape :  (15035, 19)
test_df shape :  (6468, 20)


## Catboost import

In [13]:
from lightgbm import LGBMRegressor

In [14]:
lgb_model = LGBMRegressor()
lgb_model. fit(train_X, train_y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [15]:
result = lgb_model.predict(test_df.loc[:, test_df.columns != 'id' ])

In [16]:
result = np.exp(result)

In [17]:
submission = pd.DataFrame({'id': test_df['id'],
                           'price': result})

submission.to_csv('./submission.csv', index=False)